<a href="https://colab.research.google.com/github/davemlz/eemont/blob/master/tutorials/013-Time-Series-By-Region-Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series By Region (with Conversion to Pandas and Visualization with Seaborn)

- GitHub Repo: [https://github.com/davemlz/eemont](https://github.com/davemlz/eemont)
- PyPI link: [https://pypi.org/project/eemont/](https://pypi.org/project/eemont/)
- Documentation: [https://eemont.readthedocs.io/en/0.1.8/index.html](https://eemont.readthedocs.io/en/0.1.8/index.html)
- More tutorials: [https://github.com/davemlz/eemont/tree/master/tutorials](https://github.com/davemlz/eemont/tree/master/tutorials)

## Let's start!

If required, please uncomment:

In [ ]:
#!pip install eemont
#!pip install geemap

Import the required packges.

In [ ]:
import ee, eemont, geemap
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

Authenticate and Initialize Earth Engine and geemap.

In [ ]:
Map = geemap.Map()

Let's use a buffered point for the time series!

In [ ]:
point = ee.Geometry.Point([11.178576,51.122064]).buffer(100)

Let's pre-process and process our image collection:

In [ ]:
L8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
      .filterBounds(point)
      .maskClouds()
      .scale()
      .index(['GNDVI','EVI']))

## Time Series By Region

Let's get the L8 time series for our buffer. Checklist:

- Image Collection: The Landsat 8 collection.
- Bands to use for the time series: GNDVI and EVI.
- Geometry: Our buffered point.
- Statistics to compute: Minimum, Mean, and Maximum.
- Scale: 30 m.

In [ ]:
ts = L8.getTimeSeriesByRegion(geometry = point,
                              bands = ['EVI','GNDVI'],
                              reducer = [ee.Reducer.min(),ee.Reducer.mean(),ee.Reducer.max()],
                              scale = 30)

## Conversion to Pandas

The time series is retrieved as a feature collection. To convert it to a pandas dataframe we'll use `geemap` (This may take a little bit):

In [ ]:
tsPandas = geemap.ee_to_pandas(ts)

Let's check our pandas data frame:

In [ ]:
tsPandas

What can we see here?

- The values for each band (GNDVI and EVI) are in separated columns.
- There are some -9999 values in the GNDVI and EVI columns. These values represent the NA values (e.g. Clouds or shadows). The -9999 can be changed by modifying the `naValue` parameter in the `getTimeSeriesByRegion` method (e.g. `naValue = -10000`).
- Multiple reducers can be used. In the output dataframe they are specified by a single column named `reducer`: min, mean, max.
- The date is a string that needs to be converted to a date data type.

Given this, let's curate our data frame!

First, let's get rid of the -9999 value:

In [ ]:
tsPandas[tsPandas == -9999] = np.nan

And now, let's convert the date to a date data type:

In [ ]:
tsPandas['date'] = pd.to_datetime(tsPandas['date'],infer_datetime_format = True)

We can also gather the GNDVI and EVI columns into a single column to make the data frame more 'tidy-er' (This is optional):

In [ ]:
tsPandas = pd.melt(tsPandas,
                   id_vars = ['system:index','reducer','date'],
                   value_vars = ['GNDVI','EVI'],
                   var_name = 'Index',
                   value_name = 'Value')

Let's check our curated data frame:

In [ ]:
tsPandas

## Visualization

Now, let's visualize our time series using `seaborn`:

In [ ]:
plt.figure(figsize = (20,10))
sns.lineplot(data = tsPandas,
             x = 'date',
             y = 'Value',
             hue = 'Index',
             style = 'reducer')